In [ ]:
import pandas as pd
import torch
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import torch
from gliner import GLiNER
from langchain_experimental.text_splitter import SemanticChunker
import re
from pprint import pprint
import random
from utilities import *
from chunking import EnhancedSemanticChunker
from langchain_qdrant import Qdrant
from initialize_groq import init_groq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

df = pd.read_csv("data/emails.csv")
idx = 1000
msg = df['message'][idx]
VECTOR_DB_NAME = "emails_e5_qdrant"

THIS IS TESTING

In [ ]:
print(msg)
print(clean_text(msg))
print('cuda available?',torch.cuda.is_available())

In [ ]:
# initializing Microsoft E5 model
model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
encode_kwargs = {'normalize_embeddings': True}  # does l2 norm for the cos sim
modelemb = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2", 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [ ]:
# initializing GLiNER model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gliner_model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")
gliner_model.config.max_len = 512
gliner_model.to(DEVICE)

# model entity labels configuration
labels = ["date", "location", "person", "action", "finance", "legal", "event", "product", "organization"]

In [ ]:
# initializing EnhancedSemanticChunker
enhanced_chunker = EnhancedSemanticChunker(
    embeddings=modelemb,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=50,  
    min_chunk_size=5,
    overlap_sentences=2,  
    gliner_model=gliner_model
)

In [ ]:
metadata, split_msg = extract_email_metadata(msg, idx)
msg_start = split_msg.index("X-FileName:")
full_content = clean_text(" ".join(split_msg[msg_start + 2:]))
# print(full_content)
# Create document with the enhanced chunker
documents = enhanced_chunker.create_documents(
    texts=[full_content],
    metadatas=[metadata]
)

THIS IS TESTING

In [ ]:
print(f"Number of chunks after deduplication: {len(documents)}")

# Now use 'deduplicated_documents' for further processing (indexing, etc.)
for i, doc in enumerate(documents):  
    print(f"\n--- Unique Chunk {i+1}/{len(documents)} ---")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")

In [ ]:
db = Qdrant.from_existing_collection(modelemb, "qdrant_db", VECTOR_DB_NAME)
model = modelemb

In [ ]:
prompt = ChatPromptTemplate.from_template(
        """
            Answer question based on EMAIL METADATA AND CONTENT provided. CITE YOUR SOURCES.
            {context}

            Here is question:
            {input}
        """
)

document_prompt = PromptTemplate.from_template(
    "METADATA: Source: {sender}\nDate: {date}\n Recipients: {recipient}\nSubject: {subject}\nEntities: {entities}\n\nContent: {page_content}"
)

retriever = db.as_retriever(search_kwargs={'k':20, 'search_type':'mmr','lambda_mult':0.2})

_, llm, groqllm = init_groq(model_name="llama-3.3-70b-versatile")
import random
document_chain = create_stuff_documents_chain(llm, prompt=prompt, document_prompt=document_prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Retrieve Top-K Similar Documents (Initial Broad Search)
# retriever_topk = db.as_retriever(search_kwargs={'k': 20,'fetch_k' : 100, 'search_type': 'similarity_s core_threshold','score_threshold':0.75})  # Retrieve more docs first
retriever_topk = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold':0.65,'k':10})
# MMR for Diversity (Reduce Redundant Docs)
retriever_mmr = db.as_retriever(search_type="mmr", search_kwargs={'k':10,'lambda_mult': 1})  

# Create the Hybrid Retrieval Pipeline
retrieval_chain_topk = create_retrieval_chain(retriever_topk, document_chain)  # Initial broad search
retrieval_chain_mmr = create_retrieval_chain(retriever_mmr, document_chain)    # Apply MMR re-ranking

THIS IS TESTING

In [ ]:
query = "query: is MSEB an indian company? 402 crore amount? its relation to enron?"
# pprint.pprint(retrieval_chain_topk.invoke({"input":query}))
pprint.pprint(retrieval_chain_mmr.invoke({"input":query}))

test_questions = [
    "What does randy need to send a schedule of?",
    "What are some of randy's action items?",
    "What is Philip's proposal focused on, and can you provided details about the proposal?",
    "Can you provide me more detail about the microturbine power generation deal?",
    "What needs to be faxed?",
    "Are there hints of a scandal in the emails?",
    "What did jeffrey skilling tell john arnold"
]
for text in test_questions:
    # Define query
    query = "query: " + text
    pprint.pprint(retrieval_chain_mmr.invoke({"input":query}))

test_questions = [
    "query: What does randy need to send a schedule of?",
    "query: What are some of randy's action items?",
    "query: What is Philip's proposal focused on, and can you provided details about the proposal?",
    "query: Can you provide me more detail about the microturbine power generation deal?",
    "query: What needs to be faxed?"
]
for text in test_questions:
    print("=========================================================")
    query = "query: " + text
    query_embedding = np.array(model.embed_query(query))
    # query_embedding = l2_normalize(query_embedding)  
    topk_results = db.similarity_search_with_score_by_vector(
        embedding=query_embedding.tolist(),  # List[float]
        k=5
    )

    mmr_results = db.max_marginal_relevance_search_with_score_by_vector(
        embedding=query_embedding.tolist(),  # List[float]
        k=5,
        lambda_mult=0.8         
    )

    # Sort by L2 distance (ascending: lower = more similar)
    topk_sorted = sorted(topk_results, key=lambda x: x[1])
    
    mmr_sorted = sorted(mmr_results, key=lambda x: x[1], reverse=True)

    # Display results with L2 distance and cosine similarity
    for doc, mmr_score in mmr_sorted:
        # docembedding = l2_normalize(np.array(modelemb.embed_documents([doc.page_content])))
        # cos_sim = float(np.dot(query_embedding, docembedding.reshape(-1)))
        pprint.pprint(f"Document:\n {doc.page_content} | MMR Score: {mmr_score:.4f}")
        
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    for doc, l2_score in topk_sorted:
        # Convert L2 distance to cosine similarity (assuming L2-normalized)
        cosine_sim = 1 - (l2_score ** 2) / 2
        pprint.pprint(f"Document: {doc.page_content[:100]} | L2 Distance: {l2_score:.4f} | Cosine Sim: {cosine_sim:.4f}")

In [ ]:
toolnode = ToolNode([ragtool])

memory = MemorySaver()
workflow = StateGraph(MessagesState)    
workflow.add_node("agent", call_model)
workflow.add_node(toolnode)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    router_function,
    {
       "tools": "tools",
       END: END,
    },
)
workflow.add_edge("tools", "agent")
app = workflow.compile(checkpointer=memory)

THIS IS TESTING

In [ ]:
from IPython.display import display_png
display_png(app.get_graph().draw_mermaid_png(),raw=True)

import time
while True:
    theinput = input("Enter something: ")
    if 'exit' in theinput:
        break
    inp = {"messages":[theinput]}
    
    config = {"configurable": {"thread_id": 1}}
    events = app.stream(inp, config=config, stream_mode="values")

    for event in events:
        event["messages"][-1].pretty_print()
    time.sleep(1)

In [ ]:
# Define the multi-query prompt template
# This template instructs the LLM to generate multiple search queries from a single user question
multi_template = """You are an expert at querying search engines. You specialize in understanding natural language queries and generating multiple search
queries that, taken together, would help provide a comprehensive answer to the user's question.

Main Question: {question}

Let's break this down. Generate 4 search queries for querying a knowledge store about emails. 
Make sure these queries use language that would appear in actual emails.
Remember to keep them short, using keywords that would be found in emails.
Keep them straightforward and distinct from each other.
Formulate them from different angles to solve the main query.

Return a bullet list with • at the start of each question:

• query 1
• query 2
• etc.
"""

# Create a processor to generate multiple search queries from a single question
multi_query_prompt = PromptTemplate.from_template(multi_template)
multi_query_chain = multi_query_prompt | llm | StrOutputParser()

In [ ]:
# Define test queries for evaluating retrieval performance
test_queries = [
    "What do we know about Skilling's involvement in Enron's financial reporting?",
    "What are the main topics discussed in emails from Kenneth Lay?",
    "How did Enron executives discuss the California energy crisis in their emails?",
    "What discussions were happening about LJM partnerships in the months before Enron's collapse?",
    "What was discussed about mark-to-market accounting in emails?",
    "Who was responsible for overseeing Special Purpose Entities at Enron?",
    "What communication happened regarding Raptor structures?",
]

In [ ]:
# Example usage - process a test query with advanced retrieval
for i in range(len(test_queries)):
    result = run_multi_query(test_queries[i])
    # Print the final answer
    print(result["final_answer"])